In [8]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os


In [9]:
# Define the path to the _Output directory
input_directory = "_Output"

# Initialize the images array
images = []

# Iterate through each folder in the _Output directory
for folder_name in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, folder_name)

    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # List to hold images for this folder
        folder_images = []

        # Iterate through each file in the folder
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            
            # Check if the file is an image (we'll assume all files are images)
            if os.path.isfile(file_path):
                # Read the image in grayscale
                image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

                # Add the image to the list if it's successfully read
                if image is not None:
                    folder_images.append({
                        "image_name": file,
                        "image": image
                    })

        # If the folder contains images, append the folder object to the images array
        if folder_images:
            images.append({
                "folder_name": folder_name,
                "images": folder_images
            })

# Output the result
# print(images)


# debugging
# image = cv2.imread('_Output/Mango_(P0)/0001_0003.JPG', cv2.IMREAD_GRAYSCALE)


In [10]:
def contrast_adjustment(image):
    # alpha increases contrast without changing brightness
    modified_image = cv2.convertScaleAbs(image, alpha=6, beta=-80)

    # Normalize the image to the range 0-255 for display
    modified_image = cv2.normalize(modified_image, None, 0, 255, cv2.NORM_MINMAX)

    # DEBUG
    # cv2.imshow('Equalized', modified_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return modified_image


In [11]:
def noise_reduction(image):
    modified_image = cv2.medianBlur(image, 7)

    # DEBUG
    # cv2.imshow('Blurred', modified_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return modified_image


In [12]:
# import cv2
# import numpy as np

def segmentation(image):
    # Step 1: Edge detection
    edges = cv2.Canny(image, 80, 170)

    # Step 2: Connect fragmented edges using dilation and closing
    kernel = np.ones((5, 5), np.uint8)  # Increase kernel size for better connection
    dilated_edges = cv2.dilate(edges, kernel, iterations=3)  # Increase dilation iterations
    closed_edges = cv2.morphologyEx(dilated_edges, cv2.MORPH_CLOSE, kernel, iterations=3)  # More closing iterations

    # Step 3: Detect contours from connected edges
    contours, _ = cv2.findContours(closed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Step 4: Create a mask for the connected regions
    mask = np.zeros_like(image, dtype=np.uint8)
    for contour in contours:
        # Optional: Filter small contours
        if cv2.contourArea(contour) > 500:  # Adjust minimum area as needed
            cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)

    # Step 5: Segment the inner object using the mask
    _, binary_image = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)
    segmented = cv2.bitwise_and(binary_image, binary_image, mask=mask)

    # DEBUG
    # cv2.imshow('Original Image', image)
    # cv2.imshow('Edges', edges)
    # # cv2.imshow('Dilated and Closed Edges', closed_edges)
    # cv2.imshow('Connected Regions Mask', mask)
    # cv2.imshow('Segmented Inner Object', segmented)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return segmented


In [13]:
# morphological operations
def improvement(image):
    # Create a circular kernel
    kernel_size = 5  # Adjust the size as needed
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))

    # Apply morphological opening
    modified_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=4)

    # DEBUG
    # cv2.imshow('Morphological Operations with Circular Kernel', modified_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return modified_image


In [14]:
# excution
output_directory = "segmented_output"

for folder_obj in images:
    folder_name = folder_obj['folder_name']
    folder_images = folder_obj['images']

    # Create a folder in "segmented_output" if it doesn't exist
    output_folder_path = os.path.join(output_directory, folder_name)
    os.makedirs(output_folder_path, exist_ok=True)

    # Iterate through each image in the folder
    for image_obj in folder_images:
        image_name = image_obj['image_name']
        image = image_obj['image']

        # Apply all image processing functions in sequence
        image = contrast_adjustment(image)
        image = noise_reduction(image)
        image = segmentation(image)
        image = improvement(image)

        # Save the processed image in the corresponding folder in "segmented_output"
        output_image_path = os.path.join(output_folder_path, image_name)
        cv2.imwrite(output_image_path, image)


# Debugging
# image = contrast_adjustment(image)
# image = noise_reduction(image)
# image = segmentation(image)
# image = improvement(image)
# cv2.imshow('after all', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

